# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import math
import requests
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'L(ADGALMBGCSE KATOET)RNSQ LNAES /',
 'calculationPrice': 'close',
 'open': 130.32,
 'openTime': 1678103965592,
 'openSource': 'olfficia',
 'close': 128.98,
 'closeTime': 1617773565960,
 'closeSource': 'ociifalf',
 'high': 128.41,
 'highTime': 1626487125864,
 'highSource': '  itdiemceype1r aelnd5u',
 'low': 126.54,
 'lowTime': 1687114090001,
 'lowSource': '1yp ear e5cdne iedutilm',
 'latestPrice': 130.4,
 'latestSource': 'Close',
 'latestTime': 'December 8, 2020',
 'latestUpdate': 1673333544004,
 'latestVolume': 85832793,
 'iexRealtimePrice': 126.77,
 'iexRealtimeSize': 20,
 'iexLastUpdated': 1677217621860,
 'delayedPrice': 130.45,
 'delayedPriceTime': 1626020114352,
 'oddLotDelayedPrice': 126.05,
 'oddLotDelayedPriceTime': 1631115769953,
 'extendedPrice': 125.85,
 'extendedChange': -0.1,
 'extendedChangePercent': -0.0008,
 'extendedPriceTime': 1669765432353,
 'previousClose': 124.07,
 'previousVolume': 90964342,
 'cha

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

39.3

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,118.30,54.44,N/A
1,AAL,18.46,-1.31,N/A
2,AAP,156.10,22.81,N/A
3,AAPL,130.58,39.61,N/A
4,ABBV,111.88,24.03,N/A
...,...,...,...,...
500,YUM,105.49,33.03,N/A
501,ZBH,152.66,-207.23,N/A
502,ZBRA,383.93,43.27,N/A
503,ZION,44.94,19.04,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True, drop=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,35.3800,2.05,N/A
1,NLOK,19.6800,3.8,N/A
2,BIO,585.6100,4.94,N/A
3,UNM,24.6900,5.08,N/A
4,KIM,15.1800,6.64,N/A
5,ALL,108.9400,7.69,N/A
6,MET,49.1300,7.9,N/A
7,EBAY,51.3400,8,N/A
8,BIIB,252.4400,8.42,N/A
9,PNC,145.8800,8.49,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:100000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,35.3800,2.05,56529
1,NLOK,19.6800,3.8,101626
2,BIO,585.6100,4.94,3415
3,UNM,24.6900,5.08,81004
4,KIM,15.1800,6.64,131752
5,ALL,108.9400,7.69,18358
6,MET,49.1300,7.9,40708
7,EBAY,51.3400,8,38955
8,BIIB,252.4400,8.42,7922
9,PNC,145.8800,8.49,13709


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [13]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,119.66,N/A,54.3,N/A,7.49,N/A,6.92,N/A,33.956771,N/A,13.636715,N/A,N/A
1,AAL,17.93,N/A,-1.31,N/A,-2.02,N/A,0.4492,N/A,-5.979941,N/A,1.405101,N/A,N/A
2,AAP,155.49,N/A,22.27,N/A,2.85,N/A,1.05,N/A,10.502796,N/A,2.317931,N/A,N/A
3,AAPL,126.65,N/A,39.09,N/A,33.49,N/A,8,N/A,27.511116,N/A,19.961136,N/A,N/A
4,ABBV,110.94,N/A,24.05,N/A,12.66,N/A,4.81,N/A,16.583532,N/A,9.918736,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.76,N/A,32,N/A,-4.03,N/A,5.96,N/A,22.856067,N/A,10.066617,N/A,N/A
501,ZBH,153.55,N/A,-202.39,N/A,2.62,N/A,4.4,N/A,24.681526,N/A,7.434594,N/A,N/A
502,ZBRA,377.19,N/A,43.45,N/A,10.64,N/A,4.76,N/A,29.263720,N/A,10.829349,N/A,N/A
503,ZION,45.08,N/A,18.85,N/A,1.022,N/A,2.4,N/A,10.860398,N/A,2.813191,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[pd.isnull(rv_dataframe).any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,214.840,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,233.070,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.76,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,25.340,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.720,N/A,14.42,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,57.650,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,28.998,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,134.130,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,10.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1848.740,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

In [18]:
rv_dataframe.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,119.66,N/A,54.30,0.860396,7.49,0.718812,6.9200,0.80198,33.956771,0.853465,13.636715,0.815842,N/A
1,AAL,17.93,N/A,-1.31,0.140594,-2.02,0.0435644,0.4492,0.0316832,-5.979941,0.0277228,1.405101,0.0554455,N/A
2,AAP,155.49,N/A,22.27,0.429703,2.85,0.39604,1.0500,0.146535,10.502796,0.235644,2.317931,0.120792,N/A
3,AAPL,126.65,N/A,39.09,0.718812,33.49,0.962376,8.0000,0.845545,27.511116,0.80198,19.961136,0.944554,N/A
4,ABBV,110.94,N/A,24.05,0.457426,12.66,0.859406,4.8100,0.69505,16.583532,0.520792,9.918736,0.671287,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [21]:
rv_dataframe['RV Score'] = rv_dataframe.apply(lambda row: row[[f'{ratio} Percentile' for ratio in ['PE', 'PB', 'EV/EBITDA', 'EV/GP']]].mean(), 
                                              axis=1)

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [22]:
rv_dataframe.sort_values('RV Score', inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(inplace=True, drop=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,BA,240.22,N/A,-30.79,0.0653465,-11.5000,0.0277228,2.2000,0.372277,-31.007283,0.0138614,-218.397849,0.0019802,0.027228
1,AIG,40.29,N/A,-7.04,0.10495,0.5564,0.0534653,0.7568,0.0732673,4.497819,0.039604,0.720410,0.0138614,0.052970
2,UNM,24.56,N/A,5.18,0.156436,0.4615,0.0455446,0.4125,0.029703,2.570348,0.0336634,0.402138,0.0039604,0.059901
3,AAL,17.93,N/A,-1.31,0.140594,-2.0200,0.0435644,0.4492,0.0316832,-5.979941,0.0277228,1.405101,0.0554455,0.066832
4,MET,48.43,N/A,7.79,0.162376,0.5834,0.0554455,0.6751,0.0574257,4.513586,0.0415842,0.669386,0.0118812,0.067822
5,HFC,27.09,N/A,-28.11,0.0673267,0.8201,0.0752475,0.3167,0.0138614,6.459983,0.0693069,2.210269,0.116832,0.082178
6,HIG,47.90,N/A,9.79,0.184158,0.9605,0.0891089,0.8312,0.0891089,3.844721,0.0356436,0.804229,0.019802,0.082178
7,ALL,109.95,N/A,7.71,0.160396,1.3000,0.160396,0.7207,0.0653465,2.347786,0.0316832,0.725658,0.0158416,0.092079
8,F,9.48,N/A,-17.78,0.0811881,1.1900,0.136634,0.2847,0.0118812,4.822880,0.0455446,2.145129,0.110891,0.093564
9,MRO,7.39,N/A,-9.09,0.0990099,0.5033,0.0514851,1.5000,0.242574,5.021341,0.0475248,3.163958,0.190099,0.097030


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [23]:
portfolio_input()

Enter the value of your portfolio:200000000


In [24]:
position_size = float(portfolio_size) / len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,BA,240.22,16651,-30.79,0.0653465,-11.5000,0.0277228,2.2000,0.372277,-31.007283,0.0138614,-218.397849,0.0019802,0.027228
1,AIG,40.29,99280,-7.04,0.10495,0.5564,0.0534653,0.7568,0.0732673,4.497819,0.039604,0.720410,0.0138614,0.052970
2,UNM,24.56,162866,5.18,0.156436,0.4615,0.0455446,0.4125,0.029703,2.570348,0.0336634,0.402138,0.0039604,0.059901
3,AAL,17.93,223089,-1.31,0.140594,-2.0200,0.0435644,0.4492,0.0316832,-5.979941,0.0277228,1.405101,0.0554455,0.066832
4,MET,48.43,82593,7.79,0.162376,0.5834,0.0554455,0.6751,0.0574257,4.513586,0.0415842,0.669386,0.0118812,0.067822
5,HFC,27.09,147655,-28.11,0.0673267,0.8201,0.0752475,0.3167,0.0138614,6.459983,0.0693069,2.210269,0.116832,0.082178
6,HIG,47.90,83507,9.79,0.184158,0.9605,0.0891089,0.8312,0.0891089,3.844721,0.0356436,0.804229,0.019802,0.082178
7,ALL,109.95,36380,7.71,0.160396,1.3000,0.160396,0.7207,0.0653465,2.347786,0.0316832,0.725658,0.0158416,0.092079
8,F,9.48,421940,-17.78,0.0811881,1.1900,0.136634,0.2847,0.0118812,4.822880,0.0455446,2.145129,0.110891,0.093564
9,MRO,7.39,541271,-9.09,0.0990099,0.5033,0.0514851,1.5000,0.242574,5.021341,0.0475248,3.163958,0.190099,0.097030


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [25]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [26]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [27]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [28]:
writer.save()